In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
from datetime import datetime

# старт спарк-сессии
spark = SparkSession.builder \
    .appName("TestApp") \
    .getOrCreate()

25/06/29 20:46:11 WARN Utils: Your hostname, npcserver-PC22 resolves to a loopback address: 127.0.1.1; using 192.168.0.16 instead (on interface wlo1)
25/06/29 20:46:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/29 20:46:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
# Путь к файлам
input_path = "./output_raw.csv"
second_input_path = "./output_raw_new.csv"
output_path = "./merged_cafe_data.csv"

df1 = spark.read.csv(input_path, header=True, inferSchema=True)
df2 = spark.read.csv(second_input_path, header=True, inferSchema=True)

# 1. Найти пересечение колонок
common_columns = ('name','address','rating','avg_price','reviews_count','link','lat','lon','category','insert_date')

# 2. Оставить только общие колонки в обоих датафреймах
df1_filtered = df1.select(*common_columns)
df2_filtered = df2.select(*common_columns)

# 3. Выполнить объединение
result = df1_filtered.unionByName(df2_filtered)\
        .withColumn('rating', F.regexp_replace('rating', ',', '.'))\
        .distinct()

# Убираем дубликаты и сохраняем результат в итоговый CSV-файл
result.toPandas().drop_duplicates(subset=['name', 'address']).to_csv(output_path, index=False, sep=';')

print(f"Результат сохранён в: {output_path}")


Результат сохранён в: ./merged_cafe_data.csv


In [20]:
result.where(F.col('name').contains('Pims')).count()

11

In [22]:
result.distinct().count()

22861

In [6]:
df1_filtered.show()

+--------------------+--------------------+------+---------+-------------+--------------------+---------+---------+----------+-----------+
|                name|             address|rating|avg_price|reviews_count|                link|      lat|      lon|  category|insert_date|
+--------------------+--------------------+------+---------+-------------+--------------------+---------+---------+----------+-----------+
|             Чито-Ра|      Лесная ул., 30|   4,7|     1350|         NULL|https://yandex.ru...|55.783474|37.596318|moscow_cao| 2025-06-18|
|              Пражка|Воронцовская ул.,...|   4,4|     1500|       5183.0|https://yandex.ru...|55.733954|37.662996|moscow_cao| 2025-06-18|
|           Кофемания|Садовническая ул....|   4,6|     NULL|       2619.0|https://yandex.ru...|55.734674|37.642727|moscow_cao| 2025-06-18|
|              Boston|       Лесная ул., 7|   5,0|     2250|         NULL|https://yandex.ru...|55.778629|37.587275|moscow_cao| 2025-06-18|
|         Высота 5642|Больш

In [17]:
df2.where(F.col('name') == 'Галки').show()

+-----+--------------------+------+---------+-------------+--------------------+---------+---------+--------------------+--------------------+-----------+
| name|             address|rating|avg_price|reviews_count|                link|    phone|      lat|                 lon|            category|insert_date|
+-----+--------------------+------+---------+-------------+--------------------+---------+---------+--------------------+--------------------+-----------+
|Галки|Страстной бул., 4...|   4,9|     NULL|         NULL|https://yandex.ru...|     NULL|55.764844|           37.608719|moscow_tsao_kitay...| 2025-06-22|
|Галки|Страстной бул., 4...|   4,9|     NULL|         NULL|https://yandex.ru...|     NULL|55.764844|           37.608719|moscow_tsao_kitay...| 2025-06-23|
|Галки|Страстной бул., 4...|   4,9|     1400|         NULL|https://yandex.ru...|55.764844|37.608719|moscow_svao_maryi...|          2025-06-28|       NULL|
+-----+--------------------+------+---------+-------------+-----------

In [6]:
# Путь к файлам
input_path = "./output_raw.csv"
output_path = "./merged_cafe_data.csv"

df = spark.read.csv(output_path, header=True, inferSchema=True)

In [4]:
        # ("кафе Арбат район", "moscow_tsao_arbat"),
        # ("кафе Басманный район", "moscow_tsao_basmanniy"),
        # ("кафе Замоскворечье район", "moscow_tsao_zamoskvorechye"),
        ("кафе Китай-город район", "moscow_tsao_kitay_gorod"),
        ("кафе Мещанский район", "moscow_tsao_moschanskiy"),
        ("кафе Пресненский район", "moscow_tsao_presnenskiy"),
        ("кафе Таганский район", "moscow_tsao_taganskiy"),
        ("кафе Тверской район", "moscow_tsao_tverskoy"),
        ("кафе Хамовники район", "moscow_tsao_khamovniki"),
        ("кафе Якиманка район", "moscow_tsao_yakimanka")

('кафе Якиманка район', 'moscow_tsao_yakimanka')

In [ ]:
df.where(F.col('rating') >= '4,5').count()

2519